In [4]:
import requests
import pandas as pd
import json

import os
from dotenv import load_dotenv, find_dotenv


dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

True

In [5]:
FACEIT_PLAYERID = os.environ.get("FACEIT_PLAYERID")
KEY = os.environ.get("KEY")
HEADER = {"Authorization": f"Bearer {KEY}"}
BASIC_URL = "https://open.faceit.com/data/v4/"

In [22]:

#used to retreive time registered and elo
def retreive_player_by_id(id: str) -> dict:
    response = shorter_url(url=f"players/{id}")
    return response
# used to shorted the URL and not to specify header every time
def shorter_url(url = "") -> dict:
    request = BASIC_URL + url
    return requests.get(request, headers = HEADER).json()

#gets number of games for paginaition
def get_number_of_games_played(id: str) -> int:
    response = int(shorter_url(url = f"players/{id}/stats/csgo")["lifetime"]["Matches"])
    return response

#retreived time registered
def get_time_registered(id: str) -> str:
    response = retreive_player_by_id(id)
    return response["activated_at"]


#retreived elo and level
def get_level_elo(id: str) -> int:
    response = retreive_player_by_id(id)
    return (response["games"]["csgo"]["skill_level"], response["games"]["csgo"]["faceit_elo"])

#needs paginating
# def retreive_player_game_stats(id: str) -> dict:
#     response = shorter_url(url=f"players/{id}/games/csgo/stats?offset=0&limit=20")
#     return response


def retreive_player_game_stats(id: str, offset = 0, limit = 100) -> dict:
    games_played = get_number_of_games_played(id)
    offset = 0
    times_to_run = games_played // limit
    response_array = []

    while times_to_run > 0:
        response = shorter_url(url = f"players/{id}/games/csgo/stats?offset={offset}&limit={limit}")["items"]
        response_array.extend(response)
        times_to_run-=1
        offset+=limit
        
    else:
        response = shorter_url(url = f"players/{id}/games/csgo/stats?offset={offset}&limit={limit}")["items"]
        response_array.extend(response)
    return response_array


# use for myself once only
def retreive_player_history(id: str, offset = 0, limit = 100) -> dict:
    games_played = get_number_of_games_played(id)
    registered = get_time_registered(id)
    registered = int(pd.Timestamp(registered).timestamp())
    offset = 0
    times_to_run = games_played // limit
    response_array = []

    while times_to_run > 0:
        response = shorter_url(url = f"players/{id}/history?game=csgo&from={registered}&offset={offset}&limit={limit}")["items"]
        response_array.extend(response)
        times_to_run-=1
        offset+=limit
        
    else:
        response = shorter_url(url = f"players/{id}/history?game=csgo&from={registered}&offset={offset}&limit={limit}")["items"]
        response_array.extend(response[1::])
    return response_array




In [23]:
MatchHistory = retreive_player_history(FACEIT_PLAYERID)
with open(f"../data/raw/{FACEIT_PLAYERID}_match_history.json", "w") as file:
    json.dump(MatchHistory, file )   

In [28]:
GamesStats = retreive_player_game_stats(FACEIT_PLAYERID)
with open(f"../data/raw/{FACEIT_PLAYERID}_games_stats.json", "w") as file:
    json.dump(GamesStats, file )